In [1]:
import gpt_2_simple as gpt2
import pandas as pd
import tensorflow as tf

from tqdm import tqdm
tqdm.pandas()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
metal_lyrics_df = pd.read_csv("../datasets/metal_dataset_clean_eng.csv", sep="|")

print(f"Number of songs: {len(metal_lyrics_df)}")
metal_lyrics_df.sample(10)

Number of songs: 195202


,artist,album,album_type,release_year,title,track_no,lyrics,lang_iso,lang_name
101348,Kerion,Cloudriders Part 1: Road To Skycity,Studio,2012,The Sky Is My Ocean,5,Black curtain of the night\nHas fallen on the ...,eng,English
93442,In Hearts Wake,Divination,Studio,2012,Departure (Death),3,I will stand alone to the very end. I will sta...,eng,English
173457,Theatres Des Vampires,The Vampire Chronicles,Studio,1999,Thule,3,"""By a route obscure and lonely...haunted by il...",eng,English
142919,Putridity,Ignominious Atonement,Studio,2015,The Bone Sculpture,4,Swinging changes of endocranial hypertensions\...,eng,English
128394,Nile,Festivals Of Atonement,EP,1995,Divine Intent,1,Rise in Slpender\nArms upheld unto the Sky\nBl...,eng,English
127799,Nightfall,Athenian Echoes,Studio,1995,Ishtar (Celebrate Your Beauty),3,And I've reposed all my worries\nonto thy perf...,eng,English
178160,Trap Them,Sleepwell Deconstructor,Studio,2007,Day Four: Collapse And Marathon,4,Saw the price tag you put on your eyes and ear...,eng,English
14909,As We Draw,Lines Breaking Circles,Studio,2010,Scum Of The Earth,9,"He crossed the oceans, the seas, the mountains...",eng,English
39470,Conception,The Last Sunset,Studio,1991,Another World,6,"Feeling scared, don't know why\nEveryone are s...",eng,English
157350,Shamrain,Someplace Else,Studio,2005,A Woeful Song,7,One bleak moment\nHeart ever aching\nA plainti...,eng,English


In [3]:
lyrics_corpus = "../datasets/lyrics_corpus.txt"
with open(lyrics_corpus, 'w') as out_file:
    for index, row in tqdm(metal_lyrics_df.iterrows(), total=metal_lyrics_df.shape[0]):
        out_file.write("<|startoftext|>" + str(row["lyrics"]).replace("\\n", "\n") + "<|endoftext|>" + "\n")

100%|██████████| 195202/195202 [00:25<00:00, 7805.07it/s]


In [5]:
MODEL_DIR = '../models'
MODEL_NAME = '355M'

gpt2.download_gpt2(model_dir=MODEL_DIR, model_name=MODEL_NAME)

Fetching checkpoint: 1.05Mit [00:00, 424Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 20.6Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 431Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:54, 25.9Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 315Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 20.8Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 32.9Mit/s]                                                      


In [6]:
gpt2.encode_dataset(
    "../datasets/lyrics_corpus.txt", 
    model_dir=MODEL_DIR,
    model_name=MODEL_NAME, 
    out_path="../datasets/lyrics_corpus_encoded.npz"
)

  0%|          | 0/1 [00:00<?, ?it/s]

Reading files


100%|██████████| 1/1 [05:25<00:00, 325.25s/it]


Writing ../datasets/lyrics_corpus_encoded.npz


In [ ]:
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(
    sess,
    dataset='../datasets/lyrics_corpus_encoded.npz',
    model_dir=MODEL_DIR,
    model_name=MODEL_NAME,
    steps=1000,
    restore_from='fresh',
    run_name='run1',
    print_every=10,
    sample_every=200,
    save_every=500
)